In [34]:
using Revise
using Toolips

In [35]:
using Pkg; Pkg.develop(path = "../packages/toolips/ToolipsMarkdown.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [36]:
using ToolipsMarkdown

In [37]:
using ToolipsMarkdown: TextModifier

In [38]:
tm = ToolipsMarkdown.TextStyleModifier("""
\"\"\"
# this is cool
\"\"\"
function example(x::Any)
    x = 22
end""")

ToolipsMarkdown.TextStyleModifier("\"\"\"\n# this is cool\n\"\"\"\nfunction example(x::Any)\n    x = 22\nend", Int64[], Dict{UnitRange{Int64}, Symbol}(), Dict{Symbol, Vector{Pair{String, String}}}())

In [39]:
ToolipsMarkdown.julia_block!(tm)

57:57 => :number
hello
58:58 => :number
hello


Dict{Symbol, Vector{Pair{String, String}}} with 21 entries:
  :begin       => ["color"=>"#fc038c"]
  :default     => ["color"=>"#3D3D3D"]
  :for         => ["color"=>"#fc038c"]
  :exit        => ["color"=>"teal"]
  :multistring => ["color"=>"darkgreen"]
  :mutable     => ["color"=>"teal"]
  :struct      => ["color"=>"#fc038c"]
  :func        => ["color"=>"#fc038c"]
  :module      => ["color"=>"red"]
  :string      => ["color"=>"green"]
  :in          => ["color"=>"teal"]
  :abstract    => ["color"=>"teal"]
  :char        => ["color"=>"#8b0000"]
  :end         => ["color"=>"#b81870"]
  :import      => ["color"=>"#fc038c"]
  :op          => ["color"=>"darkblue"]
  :type        => ["color"=>"#D67229"]
  :funcn       => ["color"=>"blue"]
  :number      => ["color"=>"#8b0000"]
  :if          => ["color"=>"#fc038c"]
  :using       => ["color"=>"teal"]

In [53]:
ToolipsMarkdown.mark_all!(tm, '2', :number)

2-element Vector{Vector{Int64}}:
 [57]
 [58]

In [54]:
tm.marks

Dict{UnitRange{Int64}, Symbol} with 8 entries:
  60:62 => :end
  24:31 => :func
  33:39 => :funcn
  42:46 => :type
  1:22  => :multistring
  57:57 => :number
  58:58 => :number
  57:58 => :number

In [55]:
display("text/html", string(tm))

"""
# this is cool
""" function   example (x ::Any )     x =  22 end

In [46]:
repeat_offenders = ['\n', ' ', ',', '(', ')', ';', '\"']

7-element Vector{Char}:
 '\n': ASCII/Unicode U+000A (category Cc: Other, control)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 ',': ASCII/Unicode U+002C (category Po: Punctuation, other)
 '(': ASCII/Unicode U+0028 (category Ps: Punctuation, open)
 ')': ASCII/Unicode U+0029 (category Pe: Punctuation, close)
 ';': ASCII/Unicode U+003B (category Po: Punctuation, other)
 '"': ASCII/Unicode U+0022 (category Po: Punctuation, other)

In [11]:
function mark_all!(tm::TextModifier, s::String, label::Symbol)::Nothing
    [begin
            if v[1] - 1 > 1 && ~(maximum(v) == length(tm.raw))
                if tm.raw[v[1] - 1] in repeat_offenders && tm.raw[maximum(v) + 1] in repeat_offenders
                    if ~(any(n -> n in v, tm.taken))
                        push!(tm, v => label)
                    end
                end
            elseif v[1] - 1 > 1
                if tm.raw[v[1] - 1] in repeat_offenders
                    if ~(any(n -> n in v, tm.taken))
                        push!(tm, v => label)
                    end
                end
            elseif ~(maximum(v) == length(tm.raw))
                if tm.raw[maximum(v) + 1] in repeat_offenders
                    if ~(any(n -> n in v, tm.taken))
                        push!(tm, v => label)
                    end
                end
            end  
     end for v in findall(s, tm.raw)]
    nothing
end

mark_all! (generic function with 1 method)

In [12]:
function mark_between!(tm::TextModifier, s::String, label::Symbol)
    positions::Vector{UnitRange{Int64}} = findall(s, tm.raw)
    uneven = length(positions) % 2 != 0
    [begin
        if ~(e % 2 == 0)
            if uneven && e == length(positions)
                if ~(any(n -> n in v[1]:length(tm.raw), tm.taken))
                    atstart = v[1] - 1 < 1
                    atend = maximum(v) == length(tm.raw)
                    if atstart
                        push!(tm, v[1]:length(tm.raw) => label)
                    else
                        if ~(tm.raw[maximum(mark1) + 1] == s[1])
                            push!(tm, v[1]:length(tm.raw) => label)
                        end
                    end  
                end
        else
            mark1 = minimum(v)
            mark2 = maximum(positions[e + 1])
            atstart = v[1] - 1 < 1
            atend = maximum(v) == length(tm.raw)
            if ~(atstart || atend)
                if ~(tm.raw[mark1 - 1] == s[1] || tm.raw[maximum(mark2) + 1] == s[length(s)])
                    push!(tm, mark1:mark2 => label)
                end
            elseif atstart
                if ~(tm.raw[maximum(mark2) + 1] == s[1])
                    push!(tm, mark1:mark2 => label)
                end
            elseif atend
                if tm.raw[mark1 - 1] == s[1]
                    push!(tm, mark1:mark2 => label)
                end   
            end  
        end
    end
    end for (e, v) in enumerate(positions)]
    nothing
end


mark_between! (generic function with 1 method)

In [13]:
tm.marks

Dict{UnitRange{Int64}, Symbol} with 5 entries:
  60:62 => :end
  24:31 => :func
  33:39 => :funcn
  42:46 => :type
  1:22  => :multistring

In [14]:
function mark_julia!(tm::TextModifier)
    mark_between!(tm, "\"\"\"", :multistring)
    mark_between!(tm, "\"", :string)
    mark_all!(tm, "function", :func)
    mark_all!(tm, "end", :func)
end

mark_julia! (generic function with 1 method)

In [15]:
ToolipsMarkdown.clear!(tm)

Dict{UnitRange{Int64}, Symbol}()

In [16]:
tm.raw = """
\"\"\"
# this is cool
\"\"\"
function

end

"hello!"

function example(x::Any)
    x = 5
end"""

"\"\"\"\n# this is cool\n\"\"\"\nfunction\n\nend\n\n\"hello!\"\n\nfunction example(x::Any)\n    x = 5\nend"

In [17]:
mark_julia!(tm)

In [18]:
ToolipsMarkdown.highlight_julia!(tm)

Dict{Symbol, Vector{Pair{String, String}}} with 21 entries:
  :begin       => ["color"=>"#fc038c"]
  :default     => ["color"=>"#3D3D3D"]
  :for         => ["color"=>"#fc038c"]
  :exit        => ["color"=>"teal"]
  :multistring => ["color"=>"darkgreen"]
  :mutable     => ["color"=>"teal"]
  :struct      => ["color"=>"#fc038c"]
  :func        => ["color"=>"#fc038c"]
  :module      => ["color"=>"red"]
  :string      => ["color"=>"green"]
  :in          => ["color"=>"teal"]
  :abstract    => ["color"=>"teal"]
  :char        => ["color"=>"#8b0000"]
  :end         => ["color"=>"#b81870"]
  :import      => ["color"=>"#fc038c"]
  :op          => ["color"=>"darkblue"]
  :type        => ["color"=>"#D67229"]
  :funcn       => ["color"=>"blue"]
  :number      => ["color"=>"#8b0000"]
  :if          => ["color"=>"#fc038c"]
  :using       => ["color"=>"teal"]

In [19]:
display("text/html", string(tm))

"""
# this is cool
""" function end "hello!" function  example(x::Any)     x = 5 end